In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Create a toy sine wave dataset
seq_length = 100
time_steps = np.linspace(0, np.pi, seq_length + 1)
data = np.sin(time_steps)
x = data[:-1]
y = data[1:]

# Reshape the data for PyTorch LSTM
x = x.reshape(-1, 1, 1)
y = y.reshape(-1, 1)

# Convert to PyTorch tensors
x_tensor = torch.from_numpy(x.astype(np.float32))
y_tensor = torch.from_numpy(y.astype(np.float32))

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Initialize the model
model = LSTM()

# Loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for i in range(epochs):
    for seq, labels in zip(x_tensor, y_tensor):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%10 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

# Test the model
future = 100
preds = x_tensor[0].view(1, 1, -1)
for i in range(future):
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        preds = torch.cat((preds, model(preds[-1])))

# Plot the results
plt.figure(figsize=(12,6))
plt.plot(np.arange(seq_length), data[:-1], 'r')
plt.plot(np.arange(seq_length, seq_length + future), preds[:, 0].numpy(), 'b')
plt.show()


epoch:   0 loss: 0.20908029
epoch:  10 loss: 0.02444045
epoch:  20 loss: 0.01120952
epoch:  30 loss: 0.01312835
epoch:  40 loss: 0.01385842
epoch:  50 loss: 0.01127092
epoch:  60 loss: 0.00805740
epoch:  70 loss: 0.00602375
epoch:  80 loss: 0.00240595
epoch:  90 loss: 0.00075440


RuntimeError: Tensors must have same number of dimensions: got 3 and 1